In [2]:
import os
from sys import stderr, stdout
import tempfile
import subprocess
from warnings import catch_warnings
import pandas as pd
import numpy as np
from Bio import SeqIO, SeqRecord, Seq, SearchIO, AlignIO, Phylo
from Bio.Blast import NCBIWWW, NCBIXML
import Bio.Entrez
from Bio.Phylo.TreeConstruction import DistanceCalculator,DistanceTreeConstructor
from pandas.core import construction
from pandas.core.frame import DataFrame

import re
import time


In [3]:
#Function to open fasta file of imput
def open_fasta(filename) -> SeqRecord:
    with open(filename) as handle:
        sequence_record = SeqIO.read(handle, 'fasta')
    return sequence_record

In [ ]:
#Retrieve sequence from protein id 


In [3]:
#Translation to aa
#   debugging prot
#Download prot from accession 

In [5]:
#Function to run BLAST with taxid list
def blastp_with_list(sequence, list_taxid = [], query_size = 200, blast_type = 'blastn'):
    result_handler, result_storer = None, None
    #If list is empty run query without specific taxid
    if len(list_taxid) <1:
        result_handler = NCBIWWW.qblast('blastp', 'nr', sequence, hitlist_size=query_size) 
        result_storer = result_handler.read()
    #Prepare string of Entrez and parse it to qblast
    else:
        entrez_query = ''
        for taxid in list_taxid:
            entrez_query += f'txid{taxid}[ORGN]'
            if taxid != list_taxid[-1]:
                entrez_query += ' OR '
        result_handler = NCBIWWW.qblast('blastp', 'nr', sequence, entrez_query= entrez_query, hitlist_size=query_size)
        result_storer = result_handler.read()
    return result_storer

In [6]:
#Single blast query for threading
def blastp_single_taxid(sequence, taxid, query_size = 20):
    entrez_query = f'txid{taxid}[ORGN]'
    result_handler = NCBIWWW.qblast('blastp', 'nr', sequence, entrez_query= entrez_query, hitlist_size=query_size)    
    result_storer = result_handler.read()
    return result_storer

In [7]:
#Function to create a temporary file from string and read it with SearchIO.read
def xml_string_to_handler(string):
    #make temporary file
    tmp = tempfile.NamedTemporaryFile(mode='a+')
    #write string
    tmp.write(string)
    handler = SearchIO.read(tmp.name, 'blast-xml')
    tmp.close()
    return handler

In [8]:
#Creation of a dictionary with all HSPS
#Bitscore, Bitscore raw and Evalue appear twice given that are difficult to average. For later QC only the values of the first hsp are considered
def blast_to_dictionary(blastresult):
    blast_dictionary = {'ID' : [], 'Description' : [], 'Seq_length' : [], 'Accession' : [], 'Bitscore' : [], 'Bitscore_raw' : [], 
    'Evalue' : [], 'Bitscore_combined':[], 'Bitscore_raw_combined' : [], 'Evalue_combined':[],'Hit_start' : [], 'Hit_end' : [], 'Query_frame' : [], 'Gap_num' : [], 'Aln_span' : [], 'Tot_aln_span':[], 'Identity' :[]}
    #Loop through results 
    for result in blastresult:
        blast_dictionary['ID'].append(result.id)
        blast_dictionary['Description'].append(result.description)
        blast_dictionary['Seq_length'].append(result.seq_len)
        blast_dictionary['Accession'].append(result.accession)
        #Store results of first HSP
        first_hsp = result.hsps[0]
        blast_dictionary['Bitscore'].append(first_hsp.bitscore)
        blast_dictionary['Bitscore_raw'].append(first_hsp.bitscore_raw)
        blast_dictionary['Evalue'].append(first_hsp.evalue)
        #Create variables to store results of multiple hsps
        bitscore, bitscore_raw, evalue, hitstart, hitend, queryframe, gapnum, alnspan = '','','','','','','',''
        all_alnspan, all_gapnum = [],[] 
        #Collect data of all hsps for each hit
        for hsp in result.hsps:
            bitscore += str(hsp.bitscore)
            bitscore_raw += str(hsp.bitscore_raw)
            evalue += str(hsp.evalue)
            hitstart += str(hsp.hit_start)
            hitend += str(hsp.hit_end)
            queryframe += str(hsp.query_frame)
            gapnum += str(hsp.gap_num)
            alnspan += str(hsp.aln_span)
            if hsp != result.hsps[-1]:
                bitscore += '/'
                bitscore_raw += '/' 
                evalue += '/'
                hitstart += '/'
                hitend += '/'
                queryframe += '/'
                gapnum += '/' 
                alnspan += '/' #I know it's not neat but it would give an error otherwise :(
            all_alnspan.append(int(hsp.aln_span))
            all_gapnum.append(int(hsp.gap_num))
        blast_dictionary['Bitscore_combined'].append(bitscore)
        blast_dictionary['Bitscore_raw_combined'].append(bitscore_raw)
        blast_dictionary['Evalue_combined'].append(evalue)
        blast_dictionary['Hit_start'].append(hitstart)
        blast_dictionary['Hit_end'].append(hitend)
        blast_dictionary['Query_frame'].append(queryframe)
        blast_dictionary['Gap_num'].append(gapnum)
        blast_dictionary['Aln_span'].append(alnspan)
        #Calculate total alignment span and gaps to calculate identity
        tot_alnspan, tot_gapnum = int(), int()
        seq_len = int(result.seq_len)
        for span in all_alnspan:
            tot_alnspan += span
        for gap in all_gapnum:
            tot_gapnum += gap
        identity = (tot_alnspan - tot_gapnum)/seq_len*100
        blast_dictionary['Tot_aln_span'].append(tot_alnspan)
        blast_dictionary['Identity'].append(round(identity, 3))
    return blast_dictionary

In [11]:
#Filter DF based on Evalue, sequence length and identity
def filter_df_blast(df:DataFrame, query:SeqRecord, evalue = 10**-10, difference_from_query = 50, identity_threshold=50) -> DataFrame:
    remove_index = []
    #Check if main HSP is significant for threshold, store indexes of non-significant hits
    for i in range(len(df.index)):
        eval = df.iloc[i]['Evalue']
        if float(eval) > evalue:
            remove_index.append(str(i))
        
    #Copy DF
    df_to_return = df    
    #Remove indexes if list is not empty
    if len(remove_index)>0:
        df_to_return = df_to_return.drop(df.index[int(remove_index)])
        df_to_return = df_to_return.reset_index(drop=True)
    
    #Filter by +/- % of query length
    #Obtain query length
    query_length = len(query.seq)
    #Determine upper/lower threshold of acceptance for sequences 
    lower, upper = query_length*(difference_from_query/100), query_length*((difference_from_query + 100)/100)
    #Filter DF
    df_to_return = df_to_return[df_to_return['Seq_length'] > lower]
    df_to_return = df_to_return.reset_index(drop=True)
    df_to_return = df_to_return[df_to_return['Seq_length'] < upper]
    df_to_return = df_to_return.reset_index(drop=True)

    #Filter by identity
    df_to_return = df_to_return[df_to_return['Identity'] > identity_threshold]
    df_to_return = df_to_return.reset_index(drop=True)
    
    return df_to_return

In [20]:
#Retrieve all result's entries from efetch
def retrieve_all_efetch(list_of_entries, email):
    Bio.Entrez.email = email
    list_of_sequences = []
    for entry in list_of_entries:
        handler = Bio.Entrez.efetch(db='protein', id=entry, rettype = 'fasta',retmode = 'xml', retmax=1) #Returns JSON regardless
        gb_info = Bio.Entrez.read(handler, 'text')#Returns nested lists and dictionaries 
        list_of_sequences.append(gb_info)
    return list_of_sequences

In [26]:
#Efetch to dictionary parser
def efetch_protein_to_dictionary(list_of_efetch):
    #Declare new dictionary
    dictionary = {'Accession':[],'Protein_ID':[], 'Taxid':[], 'Organism_name':[], 'Description':[], 'Seq_length':[], 'Prot_sequence':[]}
    for wrapper in list_of_efetch:
        #Cast into dictionary to avoid random exception
        result = dict(wrapper[0])
        
        acc_ver = result['TSeq_accver']
        accession = acc_ver.split('.')

        dictionary['Accession'].append(accession[0])
        dictionary['Protein_ID'].append(result['TSeq_accver'])
        dictionary['Taxid'].append(result['TSeq_taxid'])
        dictionary['Organism_name'].append(result['TSeq_orgname'])
        dictionary['Description'].append(result['TSeq_defline'])
        dictionary['Seq_length'].append(result['TSeq_length'])
        dictionary['Prot_sequence'].append(result['TSeq_sequence'])
        
    
    return dictionary


In [33]:
def filter_df_taxon(df:DataFrame, n_of_sequences = 1) -> DataFrame:
    #Make a list of all retrieved taxons
    retrieved_taxids = df['Taxid'].tolist() 
    #Make list from dictionary to eliminate duplicates 
    retrieved_taxids = list(dict.fromkeys(retrieved_taxids))
        
    #Declare empty DF
    df_toreturn = pd.DataFrame()

    #For each taxon create a DF, sort and get best result to append to df_toreturn
    for taxid in retrieved_taxids:
        #Create temporary DF exclusive to taxon
        temp_df = df[df['Taxid'] == taxid]
        temp_df = temp_df.sort_values(['Evalue', 'Identity', 'Bitscore'], ascending=[True, False, False]) #('Identity', ascending=False)
        if len(temp_df) > 0:
            if len(temp_df) > n_of_sequences:
                df_toreturn = df_toreturn.append(temp_df[:n_of_sequences])
            else:
                df_toreturn = df_toreturn.append(temp_df)

    #Refactor indexes
    df_toreturn = df_toreturn.reset_index(drop=True)

    return df_toreturn

In [56]:
#Prepare string for alingment file
def fasta_for_alignment(query:SeqRecord, df:DataFrame) -> str:
    #Initialise string and add 
    string = ''
    string += f">Query:{query.id}\n"
    string += f"{query.seq}\n"
    #Add all sequence IDs and aa sequence
    for i in range(len(df['Accession'])):
        #Check if protein is missing ------------------------------------------------Add to report?
        if len(df['Prot_sequence'][i]) <1:
            continue
        string += f">{df['Protein_ID'][i]} {df['Organism_name'][i]} {df['Description'][i]}\n"
        string += f"{df['Prot_sequence'][i]}"
        #Check if is not the last element in the list 
        if i != len(df['Accession']):
            string += '\n'
    
    return string

In [37]:
#Run mafft by saving the fasta sequences for alignment in a file and passing it to mafft 
def run_mafft_saving_file(fasta:str, mafft_directory:str, filename:str) -> str:
    file = f'{filename}.fasta'
    with open(file, 'w') as handle:
        handle.write(fasta)
    
    command_list = [mafft_directory, '--distout', f'{file}']
    process = subprocess.Popen(command_list, universal_newlines= True, stdout = subprocess.PIPE, stderr = subprocess.PIPE)
    stdout, stderr = process.communicate()

    #Save standard error of mafft
    with open('aligned_mafft_v1.stderr', 'w') as handle:
        handle.write(stderr)

    #Retrun alignment 
    return stdout


In [ ]:
#Function to get protein sequence and save fasta file given an accession number
#The function recycles functions used with lists 
def get_fasta_from_accession(accession, output_name):
    #Cast into list to recycle code 
    accession_query_list = [accession]
    #Get the efetch handler 
    query_protein_efetch = retrieve_all_efetch(accession_query_list, 'A.N.Other@example.com')
    #Make a dictionary
    dictionary_query = efetch_protein_to_dictionary(query_protein_efetch)

    #Make and save fasta file 
    fasta_string_query = f">{dictionary_query['Accession'][0]} \n {dictionary_query['Prot_sequence'][0]}"

    fasta_file_name = output_name + '_sequence.fasta'
    with open(fasta_file_name, 'w') as handle:
        handle.write(fasta_string_query)

    fasta_record_q = open_fasta(fasta_file_name)
    return fasta_record_q


In [38]:

def tree_from_alignment(alignment):
    calculator = DistanceCalculator('identity')
    distance_matrix = calculator.get_distance(alignment)

    constructor = DistanceTreeConstructor(calculator, 'nj')
    tree = constructor.build_tree(alignment)
    return tree

In [71]:
#USER INPUTS
filename = 'sry_protein.fasta' 
taxid_list = [] #['9592']
mafft_directory = r'/Users/Gioele/miniconda3/bin/mafft'
email = 'A.N.Other@example.com'
output_name = 'draft_v2_debug'


#OPTIONAL USER INPUTS
type_of_query = ['nucleotide', 'amino_acid', 'accession'] #Default amino_acid

accession = 'QBA69874'




local_query = False
threading = False
query_size = 100
server, database = '',''#FOR BLAST

evalue_threshold = 10**-10
len_threshold = 50
identity_threshold = 50

sequences_per_taxon = 3

In [44]:
#Implemented
output_df = f'{output_name}_df.csv'
output_alignment = f'{output_name}_alignment.fasta'
output_xml_tree = f'{output_name}_xml_tree.xml'
#To add
output_tree_newick = f'{output_name}_newick_tree.newick'
output_tree_jpg = f'{output_name}_tree_image.jpg'
    

In [13]:
#Open protein fasta record 
fasta_record = open_fasta(filename)

In [72]:
#Get fasta record from accession 
fasta_record = get_fasta_from_accession(accession, output_name)
fasta_record

SeqRecord(seq=Seq('MQSYASAMLSVFNSDDYSPAVQENIPALRRSSSFLCTESCNSKYQCETGENSKG...TKL'), id='QBA69874', name='QBA69874', description='QBA69874', dbxrefs=[])

In [75]:
#Get protein from nucleotide 
fasta_nucleotide = open_fasta('sry_gene.fasta') #filename 
#Further QC? 
fasta_record = fasta_nucleotide.translate(to_stop=True)

0


In [14]:
#Taxid with list
blast_results = blastp_with_list(fasta_record.seq, query_size=query_size, list_taxid=taxid_list)
blast_results

'<?xml version="1.0"?>\n<!DOCTYPE BlastOutput PUBLIC "-//NCBI//NCBI BlastOutput/EN" "http://www.ncbi.nlm.nih.gov/dtd/NCBI_BlastOutput.dtd">\n<BlastOutput>\n  <BlastOutput_program>blastp</BlastOutput_program>\n  <BlastOutput_version>BLASTP 2.12.0+</BlastOutput_version>\n  <BlastOutput_reference>Stephen F. Altschul, Thomas L. Madden, Alejandro A. Sch&amp;auml;ffer, Jinghui Zhang, Zheng Zhang, Webb Miller, and David J. Lipman (1997), &quot;Gapped BLAST and PSI-BLAST: a new generation of protein database search programs&quot;, Nucleic Acids Res. 25:3389-3402.</BlastOutput_reference>\n  <BlastOutput_db>nr</BlastOutput_db>\n  <BlastOutput_query-ID>Query_251067</BlastOutput_query-ID>\n  <BlastOutput_query-def>unnamed protein product</BlastOutput_query-def>\n  <BlastOutput_query-len>204</BlastOutput_query-len>\n  <BlastOutput_param>\n    <Parameters>\n      <Parameters_matrix>BLOSUM62</Parameters_matrix>\n      <Parameters_expect>10</Parameters_expect>\n      <Parameters_gap-open>11</Parameter

In [15]:
#Parse handler to xml_string_to_handler to be able to feed it in SearchIO.read
handler = xml_string_to_handler(blast_results)
handler

QueryResult(id='unnamed', 100 hits)

In [16]:
#Make dictionary from handler
dictionary = blast_to_dictionary(handler)

#Transfor dictionary into DF
results_df = pd.DataFrame.from_dict(dictionary)

In [17]:
#Filter DF by E-value / Bitscore / Identity
filtered_blast_df = filter_df_blast(results_df, fasta_record, evalue=evalue_threshold, difference_from_query=len_threshold, identity_threshold=identity_threshold)
   

In [18]:
#Retrieve all results from genbank
#Retrieve accession list
accession_list = filtered_blast_df['Accession'].tolist()
accession_list

['NP_003131',
 'QBA69874',
 'CAP05197',
 'CAP05199',
 'QBA69866',
 'QBA69871',
 'AFG33941',
 'QBA69864',
 'AGZ62977',
 'CAP05202',
 'QBA69863',
 'AFG33943',
 'AEK31548',
 'QBA69868',
 'AEK31551',
 'QBA69867',
 'CAP05200',
 'AGZ62970',
 'QBA69865',
 'CBH40193',
 'AEK31549',
 'QBA69872',
 'QBA69876',
 'AFG33953',
 'QBA69875',
 'AGZ62971',
 'AEK31550',
 'QBA69869',
 'AGZ62975',
 'AGZ62974',
 'P48046',
 'AEK31547',
 'AAB58342',
 'ADJ00508',
 'AAS79428',
 'AAX93513',
 'AAT37461',
 'ADJ37113',
 'ADJ37117',
 'ADJ37116',
 'ADJ00518',
 'ADJ37115',
 'ADJ00517',
 'CAD13147',
 'ADJ00513',
 'AAY88211',
 'ADJ00514',
 'ADJ00515',
 'AAT37462',
 'ADJ00511',
 'ADJ00509',
 'ADJ00512',
 'AAY15123',
 'ADJ37114',
 'NP_001008988',
 'AAX93511',
 'AAX93512',
 'Q28778',
 'Q28783',
 'ADX95829',
 'AAL09290',
 'AAT39017',
 'AGZ62972',
 'Q28447',
 'NP_001028008',
 'XP_032612415',
 'AAL09287',
 'AAS19186',
 'ADX95838',
 'AAG34391',
 'AAL09284',
 'AAL09282',
 'AAS19192',
 'ADX95840',
 'ABV44686',
 'AAW23363',
 'ABS82

In [21]:
#accession_list = ['NP_003131', 'QBA69874', 'CAP05197']
efetch_result = retrieve_all_efetch(accession_list, 'A.N.Other@example.com')

In [6]:
for result in efetch_result:
    result1 = result[0]
    print(f"Protein id {result1['TSeq_accver']}")
    print(f"Taxid {result1['TSeq_taxid']}")
    print(f"Organism name {result1['TSeq_orgname']}")
    print(f"Definition line {result1['TSeq_defline']}")
    print(f"Lenght {result1['TSeq_length']}")
    print(f"Sequence length {len(result1['TSeq_sequence'])}")
    print('\n')

Protein id NP_003131.1
Taxid 9606
Organism name Homo sapiens
Definition line sex-determining region Y protein [Homo sapiens]
Lenght 204
Sequence length 204


Protein id QBA69874.1
Taxid 9606
Organism name Homo sapiens
Definition line sex-determining region Y protein, partial [Homo sapiens]
Lenght 204
Sequence length 204


Protein id CAP05197.1
Taxid 9606
Organism name Homo sapiens
Definition line sex determining region Y [Homo sapiens]
Lenght 204
Sequence length 204




In [27]:
# Make dictionary from efetch fasta results
dictionary_efetch = efetch_protein_to_dictionary(efetch_result)

#Transfor dictionary into DF
efetch_df = pd.DataFrame.from_dict(dictionary_efetch)


In [29]:
#Merge DataFrames
left = filtered_blast_df.loc[:,['Accession', 'ID','Seq_length', 'Evalue', 'Bitscore', 'Tot_aln_span', 'Identity']]
right = efetch_df.loc[:,['Accession', 'Protein_ID', 'Taxid', 'Organism_name', 'Description', 'Prot_sequence']]
combined_df = pd.merge(left, right, on='Accession')


In [34]:
filtered_df = filter_df_taxon(combined_df, n_of_sequences=sequences_per_taxon)

In [ ]:
#Save DF of sequences that are going to be aligned 
with open(output_df, 'w') as handle:
    handle.write(filtered_df.to_csv())

In [49]:
#Prepare for multiple alingment
fasta_string = fasta_for_alignment(fasta_record, filtered_df)

In [50]:
#Run mafft alignment saving file
mafft_alignment = run_mafft_saving_file(fasta_string, mafft_directory, 'multiple_seq_fasta')

In [51]:
#Save mafft alignment
with open(output_alignment, 'w') as savefile:
    savefile.write(mafft_alignment)

In [52]:
#Open AlignIO from fasta file 
alignment = AlignIO.read(output_alignment, 'fasta')

In [53]:
#Construct tree from alignment #To change 
tree = tree_from_alignment(alignment)

In [54]:
#Write tree in xml
Phylo.write(tree, output_xml_tree, 'phyloxml')

1

In [55]:
#Draw tree
Phylo.draw_ascii(tree)


 , AAS19187.1Cercopithecus
 |
 , ABS82755.1Cercopithecus
 |
 |_ AAW23367.1Cercopithecus
 |
 | ___ AAG34440.1Allenopithecus
 ||
 ||   __ AAG34439.1Mandrillus
 ||  |
 ||  |, AAG34407.1Macaca
 || _,|
 ||| || AAG34405.1Macaca
 ||| |
 ||| |, AAG34395.1Macaca
 ||| ||
 ||| |, AAG34434.1Macaca
 ||| ||
 ||| || AAG34401.1Macaca
 ||| ||
 ||| || AAG34393.1Macaca
 |||  |
 |||  |, AAG34404.1Macaca
 |||  ||
 |||   | AAG34436.1Macaca
 |||
 |||__ NP_001028008.1Macaca
 |||
 |,|     , ADX95840.1Simias
 |||     |
 |||  ___|__ ABV44688.1Semnopithecus
 ||| |   |
 ||| |   |, ABV44686.1Presbytis
 ||| |   ,|
 ||| |   || AAG34391.1Presbytis
 ||| |   |
 ||| |   , ADX95836.1Procolobus
 ||| |   |
 ||| |   | ADX95838.1Pygathrix
 |||_|
 ||  |                _ XP_032612415.1Hylobates
 ||  |         ______|
 ||  |        |      |_ Q28447.1Hylobates
 ||  |        |
 ||  |        |        _____ AAB58342.1Carica
 ||  |        |       |
 ||  |        |       |, Query:NM_003140
 ||  |        |      ,||
 ||  |________|     